In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn import cross_validation

In [2]:
# Function to convert time into timestamp 
def convert_epoch(series):
    #final_t = series.apply(lambda x: x.replace(x.split(" ")[0], '').strip())
    final_datetime = series.apply(lambda x: datetime.strptime(x, '%d%b%Y'))
    final_datetime1 = final_datetime.apply(lambda x: x.timestamp())
    return final_datetime1

In [3]:
# Function for time in meta data
def convert_epoch_meta(series):
    #final_datetime = series.apply(lambda x: datetime.strptime(x, '%d%b%Y'))
    final_datetime1 = series.apply(lambda x: x.timestamp())
    return final_datetime1

In [4]:
# Function to correct formats of date in meta data
def try_parsing_date(text):
    for fmt in ('%d%b%Y', '%d-%B%-y', '%d-%b-%y', '%d%B%Y'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [5]:
# Data Cleaning of part1 of train data 
def data_clean(data):
    le=LabelEncoder()
    data1 = data
    data1["side"] = le.fit_transform(data1["side"])
    data1["timeofday"] = le.fit_transform(data1["timeofday"])
    data1["weekday"] = data1["date"].apply(lambda x: ep_to_day(x))
    data1["weekday"] = le.fit_transform(data1["weekday"])
    return data1

In [6]:
# Data Cleaning and preprocessing of meta data PART1
def meta_data_clean1(data):
    data1 = data
    head_id = ["ratingAgency1EffectiveDate", "ratingAgency2EffectiveDate", "maturity"]
    data1 = data1.drop(head_id, axis = 1)
    return data1

In [7]:
# Data Cleaning and Preprocessing of meta data PART2
def meta_data_clean2(data):
    data1 = data.copy()
    le=LabelEncoder()
    new_head = list(data1.columns.values)
    new_head.remove("amtIssued")
    new_head.remove("amtOutstanding")
    new_head.remove("coupon")
    new_head.remove("isin")
    new_head.remove("ratingAgency1Rating")
    new_head.remove("ratingAgency2Rating")
    new_head.remove("paymentRank")
    new_head.remove("issue date")
    new_head.remove("couponFrequency")
    data1["issue date"] = data1["issue date"].fillna("10March2016")
    #data1["issue date"] = data1["issue date"].apply(lambda x : (x.split("-")[0]+x.split("-")[1]+"20"+x.split("-")[2])
    #                                                if '-' in x else x)
    #data1["issue date"] = try_parsing_date(data1["issue date"]).values
    l = []
    for x in data1["issue date"]:
        #print(x)
        l.append(try_parsing_date(x))
    data1 = data1.drop("issue date", axis = 1)
    data1["issue date"] = l
    data1["issue date"] = convert_epoch_meta(data1["issue date"]).values
    
    for x in new_head:
        data1[x] = le.fit_transform(data1[x])
    rate_head = ["ratingAgency1Rating", "ratingAgency2Rating"]
    for x in rate_head:
        data1[x] = data1[x].apply(lambda x: int(x.replace("rating", '')))
    data1["paymentRank"] = data1["paymentRank"].apply(lambda x: int(x.replace("paymentRank", '')))
    return data1
    

In [8]:
# Function to convert timestamp to week day
import time
def ep_to_day(ep):
    return time.strftime('%a', time.localtime(ep))

In [9]:
# XGBoost Model
def run_xgboost(X, Y, Xt):    
    dTrain = xgb.DMatrix(X,label=Y,missing=np.NAN)
    dtest=xgb.DMatrix(Xt)
    XT,XV,YT,YV = cross_validation.train_test_split(X,Y,test_size=0.15,random_state=50)
    dtrain = xgb.DMatrix(XT,label=YT, missing=np.NAN)
    dval = xgb.DMatrix(XV,label=YV, missing=np.NAN)
    #param = {'eta': 0.05, 'max_depth': 8,'eval_metric': 'auc', 'objective': 'binary:logistic', 'silent': 1}

    param={
        'objective':"reg:linear",
        'booster':"gbtree",
        'eval_metric':"rmse",
        'eta':0.1,
        'max_depth':6,
        #'colsample_bytree':0.7,
        #'min_child_weight':1,
        #'num_boost_round':20,
        'silent': 1
        }

    evallist = [(dtrain,'dtrain'),(dval, 'eval')]
    num_round = 200
    bst = xgb.train(param, dTrain, num_round, evals=evallist)
    ypred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)
    ypred = pd.DataFrame(ypred)
    return ypred

## --------------------------------------------

## Starting with the dataset

In [68]:
data = pd.read_csv("dataset.csv")
meta_data = pd.read_csv("ML_Bond_metadata.csv")

In [69]:
data.head()

,isin,time,price,side,volume,timeofday,date
0,isin10033,Mon 16May16 02:51:27.000 pm,117.363618,S,730000,Afternoon,16May2016
1,isin10033,Mon 16May16 03:45:52.000 pm,115.715329,B,730000,Afternoon,16May2016
2,isin10033,Wed 6Apr16 08:24:07.000 am,115.296450,B,5000,Morning,6Apr2016
3,isin10033,Wed 6Apr16 08:24:07.000 am,115.296450,B,5000,Morning,6Apr2016
4,isin10033,Wed 6Apr16 08:25:00.000 am,115.558250,S,5000,Morning,6Apr2016


## **


First, we have clean the given dataset
Second, we have converted the given dataset to include the timeofday when on given dates trade are not done.
This we have generated and stored in train_data_gen.csv. 
So, for this code, we already had the generated file and so we are just mentioning the function by which we generated.
##### Making this train_data_gen (i.e., making data for the timeofday on given dates when no trade is done)

In [10]:
def generate_train_data_gen(data):
    m_new=data.groupby(["isin","date","side","timeofday"]).volume.sum().reset_index()
    m_backup = m_new.copy()
    m_new["date"] = convert_epoch(m_new["date"]).values
    data_cl = data_clean(m_new)
    print(data_cl.head())
    tiger = list(data_cl.groupby(["isin","date"]))
    poss_combo = [(0,1), (0,0), (1,0), (1,1)]
    len_tiger = len(tiger)

    main_isin_list = []
    print("start")
    for i in range(len_tiger):
        if i%10000 == 0:
            print(i)
        side_l = list(tiger[i][1]["side"])
        timeofday_l = list(tiger[i][1]["timeofday"])
        combo_list = list(zip(side_l, timeofday_l))
        new_side_l = []
        new_timeofday_l = []
        for combo in poss_combo:
            if combo not in combo_list:
                new_side_l.append(combo[0])
                new_timeofday_l.append(combo[1])

        len_this = len(new_side_l)
        list_df = []
        weekday_l = tiger[i][1]["weekday"].iloc[0]
        if len_this!=0:
            list_df.append(tiger[i][1])
            tmp_df = pd.DataFrame(index = np.arange(len_this))    
            tmp_df["isin"] = tiger[i][0][0]
            tmp_df["date"] = tiger[i][0][1]
            tmp_df["side"] = new_side_l
            tmp_df["timeofday"] = new_timeofday_l
            tmp_df["volume"] = 0
            #print(tiger[i][1]["weekday"])
            tmp_df["weekday"] = weekday_l
            list_df.append(tmp_df)
            main_kl = pd.concat(list_df)
            main_isin_list.append(main_kl)
            #print(main_kl)
        else:
            main_isin_list.append(tiger[i][1])


    main_isin_df = pd.concat(main_isin_list)
    print(main_isin_df.shape)
    print(main_isin_df.head())
    main_isin_df.to_csv("train_data_gen.csv")
    
# Generating train_data_gen.csv
# Here we have already generated files in our previous codes, so just putting the function here and the way to generate it
generate_train_data_gen(data)

In [11]:
main_isin_df = pd.read_csv("train_data_gen.csv")
print(main_isin_df.shape)
main_isin_df.head()

In [71]:
main_isin_df = pd.read_csv("train_data_gen.csv")
print(main_isin_df.shape)
main_isin_df.head()

main_isin_df = main_isin_df.drop("Unnamed: 0", axis = 1)

(1786652, 7)


In [72]:
main_isin_df_group = list(main_isin_df.groupby("isin"))

In [73]:
#try_12 = main_isin_df_group[0][1].copy()
#try_12.sort_values(by=['date', 'timeofday'], ascending=[True, False])

## ********************

Now, from the made train_data, we now added the dates which were not present.
#### We included the days when no trade was done for a given bond. But assuring that on those days, one of the bond was traded. This improved the accuracy.
#### This clearly shows that we have to take into account for each bond when they were not traded, as there is a high possibility for some bonds that they will not be traded on 10th, 13th and 14th June. 

In [75]:
mn = main_isin_df["date"].max()
ep_to_day(mn)
#datetime.fromtimestamp(mn)

'Thu'

In [78]:
print(main_isin_df["date"].nunique())
poss_dates = list(main_isin_df["date"].unique())
#print(poss_dates)

61


In [13]:
len_tiger = len(main_isin_df_group)
main_isin_list = []
print("start")
side_list = [0, 0, 1, 1]
time_list = [0, 1, 0, 1]
full_list_bond_df = []
for i in range(len_tiger):
    if i%10000 == 0:
        print(i)
    main_per_bondname = main_isin_df_group[i][0]
    main_per_bond = main_isin_df_group[i][1]
    weekday_l = main_per_bond["weekday"].iloc[0]
    date_list = list(main_per_bond["date"].unique())
    list_for_each_bond = []
    list_for_each_bond.append(main_per_bond)
    for x in poss_dates:
        if x not in date_list:
            tmp_df = pd.DataFrame(index = np.arange(4))
            tmp_df["isin"] = main_per_bondname
            tmp_df["date"] = x
            tmp_df["side"] = side_list
            tmp_df["timeofday"] = time_list
            tmp_df["volume"] = 0
            tmp_df["weekday"] = weekday_l
            list_for_each_bond.append(tmp_df)
    for_each_bond_df = pd.concat(list_for_each_bond)
    full_list_bond_df.append(for_each_bond_df)



In [20]:
MAIN_df_now = pd.concat(full_list_bond_df)

In [21]:
MAIN_df_now.to_csv("MAIN_DF.csv")

In [22]:
MAIN_df_now.shape

(4211684, 6)

In [23]:
MAIN_df_now.head()

,isin,date,side,timeofday,volume,weekday
0,isin0,1458239400,0,0,3900000,0
1,isin0,1458239400,1,1,7800000,0
2,isin0,1458239400,0,1,0,0
3,isin0,1458239400,1,0,0,0
4,isin0,1463077800,0,0,3870000,0


In [84]:
MAIN_df_now.shape

(4211684, 6)

In [85]:
MAIN_df_now.describe()

,date,side,timeofday,volume,weekday
count,4.211684e+06,4211684.0,4211684.0,4.211684e+06,4211684.000000
mean,1.461710e+09,0.5,0.5,3.634999e+05,2.189413
std,2.153855e+06,0.5,0.5,2.137913e+06,1.468970
min,1.458067e+09,0.0,0.0,0.000000e+00,0.000000
25%,1.459881e+09,0.0,0.0,0.000000e+00,1.000000
50%,1.461695e+09,0.5,0.5,0.000000e+00,2.000000
75%,1.463510e+09,1.0,1.0,0.000000e+00,4.000000
max,1.465411e+09,1.0,1.0,4.980400e+08,4.000000


In [88]:
MAIN_df_now["date"].max() - MAIN_df_now["date"].min()

7344000.0

In [89]:
min_date = MAIN_df_now["date"].min()
again_poss_dates = list(MAIN_df_now["date"].unique())

In [97]:
print(min_date)
MAIN_df_now["date"].max()

1458066600.0


1465410600.0

In [90]:
# MAIN_GROUP = list(MAIN_df_now.groupby("isin"))

In [105]:
# no_date = 1464546600

 ### <<< -------------- 
 ---- This was for adding the "no_date" date, i.e., "29th May" to the dataset. 
 In this day none of the bonds were traded.
 #### But adding this and then then combining with the existing framework decreased the efficiency of the model.
 So, commenting the entire section for this.
 

In [111]:

"""
len_tiger = len(MAIN_GROUP)
main_isin_list = []
print("start")
side_list = [0, 0, 1, 1]
time_list = [0, 1, 0, 1]
full_list_bond_df = []
for i in range(len_tiger):
    if i%10000 == 0:
        print(i)
    main_per_bondname = MAIN_GROUP[i][0]
    main_per_bond = MAIN_GROUP[i][1]
    weekday_l = main_per_bond["weekday"].iloc[0]
    # date_list = list(main_per_bond["date"].unique())
    list_for_each_bond = []
    list_for_each_bond.append(main_per_bond)
  
    tmp_df = pd.DataFrame(index = np.arange(4))
    tmp_df["isin"] = main_per_bondname
    tmp_df["date"] = no_date
    tmp_df["side"] = side_list
    tmp_df["timeofday"] = time_list
    tmp_df["volume"] = 0
    tmp_df["weekday"] = weekday_l
    list_for_each_bond.append(tmp_df)
    for_each_bond_df = pd.concat(list_for_each_bond)
    full_list_bond_df.append(for_each_bond_df)

"""

start
0
10000


In [112]:
# MMMAIN_df_now = pd.concat(full_list_bond_df)

In [113]:
# print(MMMAIN_df_now.shape)
# MMMAIN_df_now.to_csv("FINAL_TRAIN_DATA_PART1.csv")


(4280728, 6)


In [15]:
#MMMAIN_df_now = pd.read_csv("FINAL_TRAIN_DATA_PART1.csv")

In [16]:
#MMMAIN_df_now = MMMAIN_df_now.drop("Unnamed: 0", axis = 1)

###  ----------- >>>

### Again we saved the created MAIN_df_now dataframe to MAIN_DF.csv
### For multiple trials, we directly read it from the generated file. (If you are trying to run this notebook, just comment the next block.)
(You can check that the code for generation of this file is above.)

In [80]:
MAIN_df_now = pd.read_csv("MAIN_DF.csv")

In [81]:
MAIN_df_now.columns.values

array(['Unnamed: 0', 'isin', 'date', 'side', 'timeofday', 'volume',
       'weekday'], dtype=object)

In [82]:
MAIN_df_now = MAIN_df_now.drop("Unnamed: 0", axis = 1)

## ------------------------------------

### Now cleaning and pre processing the meta data

In [17]:
#meta_data.isnull().sum()

In [18]:
meta_data["issue date"].unique()

array([' 8Dec2011', '10-Sep-10', '14-Nov-14', ..., ' 3May2010',
       '16-Aug-10', '12-Jun-14'], dtype=object)

In [19]:
meta_data["issuer"].nunique()

4045

In [20]:

price_feature = list(data.groupby("isin"))

In [21]:
price_feature[0][1]["price"].describe()

count     5.000000
mean     83.147486
std       3.849208
min      78.801616
25%      79.063415
50%      85.870199
75%      86.001099
max      86.001099
Name: price, dtype: float64

In [22]:
price_feature[1][1].shape

(34, 7)

In [23]:
p_tmp = price_feature[5][1].copy()
p_series = convert_epoch(p_tmp["date"])

In [24]:
p_series.max()

1463682600.0

In [25]:
p_tmp1 = price_feature[5][1].copy()
aq = list(p_tmp1.groupby(["side"]))
aq[1][1]

,isin,time,price,side,volume,timeofday,date
376,isin10000,Tue 22Mar16 04:49:13.000 pm,10.121583,S,3900000,Afternoon,22Mar2016
378,isin10000,Mon 28Mar16 02:28:19.000 pm,9.738937,S,10000,Afternoon,28Mar2016
379,isin10000,Mon 28Mar16 02:29:00.000 pm,9.738937,S,10000,Afternoon,28Mar2016
380,isin10000,Tue 19Apr16 10:57:00.000 am,10.084512,S,5000,Morning,19Apr2016
384,isin10000,Thu 12May16 10:21:18.000 am,10.273008,S,5000,Morning,12May2016
386,isin10000,Mon 11Apr16 10:29:42.000 am,9.749409,S,5000,Morning,11Apr2016


In [26]:
meta_data_1 = meta_data_clean1(meta_data)
meta_data_1.head()

,isin,issuer,issue date,Market,amtIssued,amtOutstanding,collateralType,coupon,couponFrequency,couponType,...,industrySector,industrySubgroup,maturityType,securityType,paymentRank,144aFlag,ratingAgency1Rating,ratingAgency1Watch,ratingAgency2Rating,ratingAgency2Watch
0,isin0,issuer490,8Dec2011,Market2,183260000,183260000,collateralType19,10.000,2.0,couponType1,...,industrySector0,industrySubgroup146,maturityType5,securityType3,paymentRank3,flag0,rating0,watch0,rating0,watch0
1,isin1,issuer1193,10-Sep-10,Market2,261800000,198970000,collateralType26,5.500,2.0,couponType1,...,industrySector0,industrySubgroup146,maturityType10,securityType5,paymentRank8,flag0,rating0,watch0,rating0,watch0
2,isin2,issuer2943,14-Nov-14,Market2,78540000,78540000,collateralType28,5.750,2.0,couponType1,...,industrySector0,industrySubgroup239,maturityType10,securityType5,paymentRank9,flag1,rating0,watch0,rating0,watch0
3,isin3,issuer3707,4Mar2013,Market1,1570800000,1570800000,collateralType28,4.750,1.0,couponType6,...,industrySector6,industrySubgroup85,maturityType5,securityType2,paymentRank9,flag0,rating18,watch0,rating22,watch0
4,isin4,issuer3711,25-Nov-14,Market0,1309000000,1309000000,collateralType12,7.125,1.0,couponType6,...,industrySector6,industrySubgroup85,maturityType13,securityType3,paymentRank5,flag0,rating15,watch0,rating0,watch0


In [27]:
meta_data_2 = meta_data_clean2(meta_data_1)

In [30]:
meta_data_2["issuer"].nunique()

4045

In [31]:
epoch_test = datetime.strptime("10Jun2016", '%d%b%Y')
epoch_test = epoch_test.timestamp()
def time_last_delta(t1):
    x = datetime.fromtimestamp(epoch_test)
    y = datetime.fromtimestamp(t1)
    z = x - y
    return z.days

#### Adding features like mean_price, standard_deviation_price, number of days bond is traded, total price sell, total price buy.


In [32]:
map_mean = {}
map_std = {}
map_numdays = {}
map_day_std = {}
map_sell_num = {}
map_buy_num = {}

map_buy_vol = {}
map_sell_vol = {}

for i in range(meta_data.shape[0]):
    bond_num = price_feature[i][0]
    meanx = price_feature[i][1]["price"].mean()
    stdx = price_feature[i][1]["price"].std()
    
    # vol_mean = price_feature[i][1]["volume"].sum()
    map_mean[bond_num] = meanx
    map_std[bond_num] = stdx
    map_numdays[bond_num] = price_feature[i][1].shape[0]
    
    p_series = convert_epoch(price_feature[i][1]["date"])
    last_epoch1 = p_series.std()
    #print(last_epoch1)
    map_day_std[bond_num] = last_epoch1
    
    p_tmp1 = price_feature[i][1].copy()
    p_tmp_group = list(p_tmp1.groupby("side"))
    #print(p_tmp_group)
    if len(p_tmp_group) == 2:
        map_sell_num[bond_num] = p_tmp_group[1][1].shape[0]
        cat_sell = p_tmp_group[1][1]["volume"].sum()
        map_buy_num[bond_num] = p_tmp_group[0][1].shape[0]
        cat_buy = p_tmp_group[0][1]["volume"].sum()
    else:
        if p_tmp_group[0][0] == 0:
            map_buy_num[bond_num] = p_tmp_group[0][1].shape[0]
            map_sell_num[bond_num] = 0
            cat_buy = p_tmp_group[0][1]["volume"].sum()
            cat_sell = 0
        else:
            map_sell_num[bond_num] = p_tmp_group[0][1].shape[0]
            map_buy_num[bond_num] = 0
            cat_sell = p_tmp_group[0][1]["volume"].sum()
            cat_buy = 0
            
    map_buy_vol[bond_num] = meanx * cat_buy
    map_sell_vol[bond_num] = meanx * cat_sell

mean_col = []
std_col = []
trade_days_col = []
days_std_col = []

days_sell = []
days_buy = []

cat_sell = []
cat_buy = []

for bond in meta_data_2["isin"]:
    mean_col.append(map_mean[bond])
    std_col.append(map_std[bond])
    trade_days_col.append(map_numdays[bond])
    days_std_col.append(map_day_std[bond])
    days_sell.append(map_sell_num[bond])
    days_buy.append(map_buy_num[bond])
    cat_sell.append(map_sell_vol[bond])
    cat_buy.append(map_buy_vol[bond])

In [33]:
meta_data_2["price_mean"] = mean_col
meta_data_2["price_std"] = std_col
meta_data_2["trade_days"] = trade_days_col

meta_data_2["cat_sell"] = cat_sell
meta_data_2["cat_buy"] = cat_buy
# meta_data_2["days_std"] = days_std_col
# meta_data_2["buy_days"] = days_buy
# meta_data_2["sell_days"] = days_sell
meta_data_2_tmp = meta_data_2.copy()

In [38]:
meta_data_2.to_csv("meta_wihout_mat.csv",index=False)

## ------------------------------------------------

### Merging the data and the metadata together now

In [83]:
mrg_data = MAIN_df_now.merge(meta_data_2,on='isin')

In [84]:
mrg_data.head()

,isin,date,side,timeofday,volume,weekday,issuer,Market,amtIssued,amtOutstanding,...,ratingAgency1Rating,ratingAgency1Watch,ratingAgency2Rating,ratingAgency2Watch,issue date,price_mean,price_std,trade_days,cat_sell,cat_buy
0,isin0,1.458239e+09,0,0,3900000,0,3480,2,183260000,183260000,...,0,0,0,0,1.323283e+09,83.147486,3.849208,5,9.703312e+08,6.460560e+08
1,isin0,1.458239e+09,1,1,7800000,0,3480,2,183260000,183260000,...,0,0,0,0,1.323283e+09,83.147486,3.849208,5,9.703312e+08,6.460560e+08
2,isin0,1.458239e+09,0,1,0,0,3480,2,183260000,183260000,...,0,0,0,0,1.323283e+09,83.147486,3.849208,5,9.703312e+08,6.460560e+08
3,isin0,1.458239e+09,1,0,0,0,3480,2,183260000,183260000,...,0,0,0,0,1.323283e+09,83.147486,3.849208,5,9.703312e+08,6.460560e+08
4,isin0,1.463078e+09,0,0,3870000,0,3480,2,183260000,183260000,...,0,0,0,0,1.323283e+09,83.147486,3.849208,5,9.703312e+08,6.460560e+08


In [85]:
mrg_data["diff_issue_date"] = mrg_data["date"] - mrg_data["issue date"] 
mrg_data = mrg_data.drop("issue date", axis = 1)
mrg_data.head()

,isin,date,side,timeofday,volume,weekday,issuer,Market,amtIssued,amtOutstanding,...,ratingAgency1Rating,ratingAgency1Watch,ratingAgency2Rating,ratingAgency2Watch,price_mean,price_std,trade_days,cat_sell,cat_buy,diff_issue_date
0,isin0,1.458239e+09,0,0,3900000,0,3480,2,183260000,183260000,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,134956800.0
1,isin0,1.458239e+09,1,1,7800000,0,3480,2,183260000,183260000,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,134956800.0
2,isin0,1.458239e+09,0,1,0,0,3480,2,183260000,183260000,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,134956800.0
3,isin0,1.458239e+09,1,0,0,0,3480,2,183260000,183260000,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,134956800.0
4,isin0,1.463078e+09,0,0,3870000,0,3480,2,183260000,183260000,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,139795200.0


In [86]:
mrg_data.shape

(4211684, 31)

In [87]:
train_Y = mrg_data["volume"]
drop_col = ["isin", "volume"]
train_X = mrg_data.drop(drop_col, axis = 1)

## -----------------------------------------------------------------

### Creating the test dataset for 10th, 13th, 14th June.

In [42]:
dates = []
date_epoch = []
dates.append("10Jun2016")
dates.append("13Jun2016")
dates.append("14Jun2016")
for x in dates:
    epoch = datetime.strptime(x, '%d%b%Y')
    date_epoch.append(epoch.timestamp())
date_epoch = date_epoch + date_epoch
date_epoch = date_epoch + date_epoch
side = [0, 0, 0, 1, 1, 1]
side = side + side
timeofday = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
len(date_epoch)

12

In [43]:
bond_num = list(meta_data["isin"])
head_tmp = ["isin", "date", "side", "timeofday"]
df_list = []
for x in bond_num:
    tmp_df = pd.DataFrame(index = np.arange(12))
    tmp_df["isin"] = x
    tmp_df["date"] = date_epoch
    tmp_df["side"] = side
    tmp_df["timeofday"] = timeofday
    df_list.append(tmp_df)
main_df = pd.concat(df_list)
test = main_df

In [44]:
le = LabelEncoder()
test["weekday"] = test["date"].apply(lambda x: ep_to_day(x))
test["weekday"] = le.fit_transform(test["weekday"])

In [45]:
mrg_test_data = test.merge(meta_data_2_tmp,on='isin')
mrg_test_data["diff_issue_date"] = mrg_test_data["date"] - mrg_test_data["issue date"] 
mrg_test_data = mrg_test_data.drop("issue date", axis = 1)
mrg_test_data.head()

,isin,date,side,timeofday,weekday,issuer,Market,amtIssued,amtOutstanding,collateralType,...,ratingAgency1Rating,ratingAgency1Watch,ratingAgency2Rating,ratingAgency2Watch,price_mean,price_std,trade_days,cat_sell,cat_buy,diff_issue_date
0,isin0,1.465497e+09,0,0,0,3480,2,183260000,183260000,11,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,142214400.0
1,isin0,1.465756e+09,0,0,1,3480,2,183260000,183260000,11,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,142473600.0
2,isin0,1.465843e+09,0,0,2,3480,2,183260000,183260000,11,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,142560000.0
3,isin0,1.465497e+09,1,0,0,3480,2,183260000,183260000,11,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,142214400.0
4,isin0,1.465756e+09,1,0,1,3480,2,183260000,183260000,11,...,0,0,0,0,83.147486,3.849208,5,9.703312e+08,6.460560e+08,142473600.0


In [46]:
test_X = mrg_test_data.drop("isin", axis = 1)

In [47]:
test_X.columns.values

array(['date', 'side', 'timeofday', 'weekday', 'issuer', 'Market',
       'amtIssued', 'amtOutstanding', 'collateralType', 'coupon',
       'couponFrequency', 'couponType', 'industryGroup', 'industrySector',
       'industrySubgroup', 'maturityType', 'securityType', 'paymentRank',
       '144aFlag', 'ratingAgency1Rating', 'ratingAgency1Watch',
       'ratingAgency2Rating', 'ratingAgency2Watch', 'price_mean',
       'price_std', 'trade_days', 'cat_sell', 'cat_buy', 'diff_issue_date'], dtype=object)

## --------------------------------------------------

### XGBoost Model

In [88]:
ypred = run_xgboost(train_X, train_Y, test_X)


[0]	dtrain-rmse:2.1341e+06	eval-rmse:2.11612e+06
[1]	dtrain-rmse:2.10343e+06	eval-rmse:2.08613e+06
[2]	dtrain-rmse:2.07802e+06	eval-rmse:2.06128e+06
[3]	dtrain-rmse:2.05717e+06	eval-rmse:2.04077e+06
[4]	dtrain-rmse:2.03997e+06	eval-rmse:2.02383e+06
[5]	dtrain-rmse:2.02567e+06	eval-rmse:2.00959e+06
[6]	dtrain-rmse:2.01356e+06	eval-rmse:1.99801e+06
[7]	dtrain-rmse:2.00381e+06	eval-rmse:1.98816e+06
[8]	dtrain-rmse:1.99505e+06	eval-rmse:1.97984e+06
[9]	dtrain-rmse:1.98844e+06	eval-rmse:1.97306e+06
[10]	dtrain-rmse:1.98205e+06	eval-rmse:1.96699e+06
[11]	dtrain-rmse:1.97638e+06	eval-rmse:1.96173e+06
[12]	dtrain-rmse:1.97091e+06	eval-rmse:1.9567e+06
[13]	dtrain-rmse:1.96716e+06	eval-rmse:1.95284e+06
[14]	dtrain-rmse:1.96385e+06	eval-rmse:1.94964e+06
[15]	dtrain-rmse:1.96092e+06	eval-rmse:1.94689e+06
[16]	dtrain-rmse:1.95817e+06	eval-rmse:1.94441e+06
[17]	dtrain-rmse:1.9553e+06	eval-rmse:1.94104e+06
[18]	dtrain-rmse:1.95314e+06	eval-rmse:1.93912e+06
[19]	dtrain-rmse:1.95078e+06	eval-rmse:1.936

### test.head()

In [89]:
ypred_pos = ypred.copy()
ypred_pos[0] = ypred_pos[0].apply(lambda x: 0.0 if x<0 else x)

In [90]:
new_test_rew = test.copy()
new_test_rew.shape
ypred.shape
new_test_rew["volume"] = ypred_pos[0].values

In [91]:
ypred[0].nunique()

202516

In [92]:
new_test_rew["volume"].unique()

array([  78187.484375  ,   99500.6875    ,  120156.3359375 , ...,
         67137.6015625 ,   19414.28125   ,   15884.38964844])

In [93]:
ftd_group_bond = list(new_test_rew.groupby("isin"))

In [94]:
ftd_group_bond_side = list(ftd_group_bond[0][1].groupby("side"))

In [95]:
ftd_group_bond_side[1][1]["isin"]

3     isin0
4     isin0
5     isin0
9     isin0
10    isin0
11    isin0
Name: isin, dtype: object

## -------------------------------------------

### The model predicts for each day and each timeofday for sell and buy differently. So, summing it up and making it into the required format.

In [96]:
new_test_rew.shape
len(ftd_group_bond)

17261

In [97]:
test_qwerty = pd.DataFrame(index = np.arange(17261))

In [98]:
col1 = []
col2 = []
col3 = []

In [99]:
for i in range(17261):
    bond_num = ftd_group_bond[i][0]
    ftd_group_bond_side = list(ftd_group_bond[i][1].groupby("side"))
    t_buy = ftd_group_bond_side[0][1] #Buy
    t_sell = ftd_group_bond_side[1][1] #Sell
    buy_vol = t_buy["volume"].sum()
    sell_vol = t_sell["volume"].sum()
    col1.append(bond_num)
    col2.append(buy_vol)
    col3.append(sell_vol)

In [100]:
len(col3)

17261

In [101]:
test_qwerty["isin"] = col1
test_qwerty["buyvolume"] = col2
test_qwerty["sellvolume"] = col3

In [102]:
test_qwerty.head()

,isin,buyvolume,sellvolume
0,isin0,537858.769531,5.610202e+05
1,isin1,924086.744141,1.132882e+06
2,isin10,187615.764648,1.787654e+05
3,isin100,174825.561523,1.708702e+05
4,isin1000,105074.780273,9.663609e+04


In [103]:
test_qwerty["buyvolume"].unique()

array([ 537858.76953125,  924086.74414062,  187615.76464844, ...,
        134822.0447998 ,  182625.96582031,  143891.30126953])

In [104]:
test_qwerty.to_csv("SOL_MAIN_TRY_LION.csv", index = False)

In [105]:
ypred[0].nunique()

202516